<a href="https://colab.research.google.com/github/vsvale/Apache-PySpark-by-Example/blob/main/Apache_PySpark_by_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [1]:
%%capture
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Download Dataset

Este conjunto de dados reflete incidentes de crime relatados (com exceção de assassinatos em que existem dados para cada vítima) que ocorreram na cidade de Chicago de 2001 até o presente, menos os sete dias mais recentes. Os dados são extraídos do sistema CLEAR (Citizen Law Enforcement Analysis and Reporting) do Departamento de Polícia de Chicago. Para proteger a privacidade das vítimas de crimes, os endereços são mostrados apenas no nível do bloco e os locais específicos não são identificados.

In [3]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2021-04-11 18:51:56--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [        <=>         ]   1.60G  3.21MB/s    in 8m 19s  

2021-04-11 19:00:16 (3.29 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1723679791]



In [4]:
!mv /content/rows.csv?accessType=DOWNLOAD /content/reported_crimes.csv

# Create Dataframe

In [5]:
#criar o dataframe spark a partir do dataset
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('./reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a'))

In [6]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

# Limit example

In [103]:
#selecionar apenas algumas linhas
small_set = rc.limit(100)
small_set.count()

100

In [104]:
small_set.show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|Community Area|FBI Code|Year|          Updated On| DISTRICT NAME|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|  12|            61|     08B|2015|02/10/2018 03:50:...|       Deering|
|10224739|   HY411615|2015-09-04 11:30:00| 008XX N CENTRAL AVE|0870|             THEFT|      POCKET-PICKING|        

In [105]:
del small_set

# Schema

In [9]:
rc.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Location', 'string')]

In [10]:
#Spark define schema com base em algumas linhas
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [11]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

# Define Schema

In [12]:
#definir manualmente garante a qualidade do Schema
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, DoubleType, IntegerType, TimestampType
schema = StructType([
            #Campo('nomedocampo',TipodoCampo,Aceita_NUll),
            StructField('ID', StringType(),True),
            StructField('Case Number',StringType(),True),
            StructField('Date',TimestampType(),True),
            StructField('Block',StringType(),True),
            StructField('IUCR',StringType(),True),
            StructField('Primary Type',StringType(),True),
            StructField('Description',StringType(),True),
            StructField('Location Description',StringType(),True),
            StructField('Arrest',StringType(),True),
            StructField('Domestic',BooleanType(),True),
            StructField('Beat',StringType(),True),
            StructField('District',StringType(),True),
            StructField('Ward',StringType(),True),
            StructField('Community Area',StringType(),True),
            StructField('FBI Code',StringType(),True),
            StructField('X Coordinate',StringType(),True),
            StructField('Y Coordinate',StringType(),True),
            StructField('Year',IntegerType(),True),
            StructField('Updated On',StringType(),True),
            StructField('Latitude',DoubleType(),True),
            StructField('Longitude',DoubleType(),True),
            StructField('Location',StringType(),True)
])
schema

StructType(List(StructField(ID,StringType,true),StructField(Case Number,StringType,true),StructField(Date,TimestampType,true),StructField(Block,StringType,true),StructField(IUCR,StringType,true),StructField(Primary Type,StringType,true),StructField(Description,StringType,true),StructField(Location Description,StringType,true),StructField(Arrest,StringType,true),StructField(Domestic,BooleanType,true),StructField(Beat,StringType,true),StructField(District,StringType,true),StructField(Ward,StringType,true),StructField(Community Area,StringType,true),StructField(FBI Code,StringType,true),StructField(X Coordinate,StringType,true),StructField(Y Coordinate,StringType,true),StructField(Year,IntegerType,true),StructField(Updated On,StringType,true),StructField(Latitude,DoubleType,true),StructField(Longitude,DoubleType,true),StructField(Location,StringType,true)))

In [13]:
rc = spark.read.csv('reported_crimes.csv', schema=schema)

In [14]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [15]:
rc.show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|    null|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|nul

In [16]:
del rc

# Select Columns

In [17]:
rc = spark.read.csv('./reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a'))

In [18]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [19]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [20]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [21]:
rc.select('Case Number','Date','Arrest').show(5)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HY411648|2015-09-05 13:30:00| false|
|   HY411615|2015-09-04 11:30:00| false|
|   JC213529|2018-09-01 00:01:00| false|
|   HY411595|2015-09-05 12:45:00|  true|
|   HY411610|2015-09-05 13:00:00| false|
+-----------+-------------------+------+
only showing top 5 rows



# Literal

In [22]:
from pyspark.sql.functions import lit
rc = rc.withColumn('One',lit(1))

In [23]:
rc.select('Case Number','Date','Arrest','One').show(5)

+-----------+-------------------+------+---+
|Case Number|               Date|Arrest|One|
+-----------+-------------------+------+---+
|   HY411648|2015-09-05 13:30:00| false|  1|
|   HY411615|2015-09-04 11:30:00| false|  1|
|   JC213529|2018-09-01 00:01:00| false|  1|
|   HY411595|2015-09-05 12:45:00|  true|  1|
|   HY411610|2015-09-05 13:00:00| false|  1|
+-----------+-------------------+------+---+
only showing top 5 rows



# Drop Column

In [24]:
rc = rc.drop('One')
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

# Add Row

In [25]:
from datetime import datetime

In [26]:
#lista com novos dados
new_row = [['20000055','HY189684',datetime.now(),'032XX W HURON ST','0560','ROBBERY','ARMED: HANDGUN','RESIDENCE',True,True,'2523','002','4','40','08B','1154760','1928502',2021,'04/04/2021 19:51:52',41.893720636,-87.622361533,'(41.893720636,-87.622361533)']]
#lista para spark dataframe
new_df = spark.createDataFrame(new_row,schema=rc.schema)
new_df.show(1)

+--------+-----------+--------------------+----------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|           Block|IUCR|Primary Type|   Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|         Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+----------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+------------+-------------+--------------------+
|20000055|   HY189684|2021-04-11 19:00:...|032XX W HURON ST|0560|     ROBBERY|ARMED: HANDGUN|           RESIDENCE|  true|    true|2523|     002|   4|            40|     08B|     115

In [27]:
#unir linha nova ao restante dos dados
rc = rc.union(new_df)

In [28]:
#filtrar apenas a nova linha pelo id
rc.filter(col('ID') == lit('20000055')).show()

+--------+-----------+--------------------+----------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|           Block|IUCR|Primary Type|   Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|         Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+----------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+------------+-------------+--------------------+
|20000055|   HY189684|2021-04-11 19:00:...|032XX W HURON ST|0560|     ROBBERY|ARMED: HANDGUN|           RESIDENCE|  true|    true|2523|     002|   4|            40|     08B|     115

In [29]:
#Ordernar de forma decrescente para ver a nova linha e o restante dos dados
rc.orderBy('ID',ascending=False).show(5)

+-------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|9999999|   HY189683|2015-03-18 17:08:00| 043XX N CENTRAL AVE|0560|     ASSAULT|              SIMPLE|          RESTAURANT|  true|   false|1624|     016|  

# Top 10 crimes por tipo

In [30]:
rc.groupBy('Primary Type').count().orderBy('count',ascending=False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1538553|
|            BATTERY|1337949|
|    CRIMINAL DAMAGE| 831752|
|          NARCOTICS| 737218|
|            ASSAULT| 464133|
|      OTHER OFFENSE| 453448|
|           BURGLARY| 409215|
|MOTOR VEHICLE THEFT| 336770|
| DECEPTIVE PRACTICE| 310298|
|            ROBBERY| 274557|
+-------------------+-------+
only showing top 10 rows



# Percentual de crimes reportados que resultaram em prisão

In [31]:
rc.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [32]:
((rc.filter(col('Arrest') == 'true').count())/rc.count())*100

27.065234531992786

# Top 3 lugares para sofrer um crime

In [33]:
rc.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1896337|
|           RESIDENCE|1241265|
|           APARTMENT| 784397|
+--------------------+-------+
only showing top 3 rows



# Top 3 quarteirões dos narcoticos

In [34]:
rc.filter(col('Primary Type')==lit('NARCOTICS')).groupBy('Block').count().orderBy('count',ascending=False).show(3)

+--------------------+-----+
|               Block|count|
+--------------------+-----+
| 033XX W FILLMORE ST| 3081|
|005XX E BROWNING AVE| 1706|
|038XX W ROOSEVELT RD| 1613|
+--------------------+-----+
only showing top 3 rows



# O dia para ficar em casa

In [35]:
rc.groupBy('Date').count().orderBy('count',ascending=False).show(1)

+-------------------+-----+
|               Date|count|
+-------------------+-----+
|2008-01-01 00:01:00|  303|
+-------------------+-----+
only showing top 1 row



# Some functions

In [36]:
from pyspark.sql.functions import lower,upper,substring,concat

In [37]:
rc.select(lower(col('Primary Type'))).show(5)

+-------------------+
|lower(Primary Type)|
+-------------------+
|            battery|
|              theft|
|              theft|
|          narcotics|
|            assault|
+-------------------+
only showing top 5 rows



In [38]:
rc.select(upper(col('Arrest'))).show(5)

+-------------+
|upper(Arrest)|
+-------------+
|        FALSE|
|        FALSE|
|        FALSE|
|         TRUE|
|        FALSE|
+-------------+
only showing top 5 rows



In [39]:
rc.select(substring(col('Description'),1,4)).show(5)

+----------------------------+
|substring(Description, 1, 4)|
+----------------------------+
|                        DOME|
|                        POCK|
|                        OVER|
|                        POSS|
|                        SIMP|
+----------------------------+
only showing top 5 rows



In [40]:
rc.select(lower(col('Primary Type')),upper(col('Arrest')),substring(col('Description'),1,4)).show(5)

+-------------------+-------------+----------------------------+
|lower(Primary Type)|upper(Arrest)|substring(Description, 1, 4)|
+-------------------+-------------+----------------------------+
|            battery|        FALSE|                        DOME|
|              theft|        FALSE|                        POCK|
|              theft|        FALSE|                        OVER|
|          narcotics|         TRUE|                        POSS|
|            assault|        FALSE|                        SIMP|
+-------------------+-------------+----------------------------+
only showing top 5 rows



In [41]:
rc.select(concat(lower(substring(col('Primary Type'),1,3)),lower(substring(col('Location Description'),3,6))).alias("random_word")).show(5)

+-----------+
|random_word|
+-----------+
|  batsidenc|
|   thea bus|
|  thesidenc|
|  nardewalk|
|  assartmen|
+-----------+
only showing top 5 rows



In [42]:
from pyspark.sql.functions import min,max

In [43]:
rc.select(min(col('Date')),max(col('Date'))).show()

+-------------------+--------------------+
|          min(Date)|           max(Date)|
+-------------------+--------------------+
|2001-01-01 00:00:00|2021-04-11 19:00:...|
+-------------------+--------------------+



In [44]:
from pyspark.sql.functions import date_add,date_sub

In [45]:
rc.select(date_sub(min(col('Date')),3).alias("Min_date-3"),date_add(max(col('Date')),3).alias("Max_date+3")).show()

+----------+----------+
|Min_date-3|Max_date+3|
+----------+----------+
|2000-12-29|2021-04-14|
+----------+----------+



# Date

In [46]:
from pyspark.sql.functions import to_date, to_timestamp, date_format

In [47]:
#Parse
rc.select(col('Date'), \
          to_date(col('Date'),'yyyy-MM-dd HH:mm:ss'), \
          to_timestamp(col('Date'),'yyyy-MM-dd HH:mm:ss'), \
          ).show()

+-------------------+----------------------------------+---------------------------------------+
|               Date|to_date(Date, yyyy-MM-dd HH:mm:ss)|to_timestamp(Date, yyyy-MM-dd HH:mm:ss)|
+-------------------+----------------------------------+---------------------------------------+
|2015-09-05 13:30:00|                        2015-09-05|                    2015-09-05 13:30:00|
|2015-09-04 11:30:00|                        2015-09-04|                    2015-09-04 11:30:00|
|2018-09-01 00:01:00|                        2018-09-01|                    2018-09-01 00:01:00|
|2015-09-05 12:45:00|                        2015-09-05|                    2015-09-05 12:45:00|
|2015-09-05 13:00:00|                        2015-09-05|                    2015-09-05 13:00:00|
|2015-09-05 10:55:00|                        2015-09-05|                    2015-09-05 10:55:00|
|2015-09-04 18:00:00|                        2015-09-04|                    2015-09-04 18:00:00|
|2015-09-05 13:00:00|         

In [48]:
#Format
rc.select(col('Date'), \
          date_format(col('Date'),'G').alias('Era'), \
          date_format(col('Date'),'y').alias('Year'), \
          date_format(col('Date'),'M').alias('Month in year'), \
          date_format(col('Date'),'D').alias('Day in year'), \
          date_format(col('Date'),'d').alias('Day in month'), \
          date_format(col('Date'),'E').alias('Day name in week'), \
          date_format(col('Date'),'a').alias('AM/PM'), \
          date_format(col('Date'),'H').alias('Hour (0-23)'), \
          date_format(col('Date'),'k').alias('Hour (1-24)'), \
          date_format(col('Date'),'K').alias('Hour AM/PM (0-11)'), \
          date_format(col('Date'),'h').alias('Hour AM/PM (1-12)'), \
          date_format(col('Date'),'m').alias('Minute'), \
          date_format(col('Date'),'s').alias('Second'), \
          date_format(col('Date'),'S').alias('milisecond'), \
          date_format(col('Date'),'z').alias('Timezone'), \
          ).show()

+-------------------+---+----+-------------+-----------+------------+----------------+-----+-----------+-----------+-----------------+-----------------+------+------+----------+--------+
|               Date|Era|Year|Month in year|Day in year|Day in month|Day name in week|AM/PM|Hour (0-23)|Hour (1-24)|Hour AM/PM (0-11)|Hour AM/PM (1-12)|Minute|Second|milisecond|Timezone|
+-------------------+---+----+-------------+-----------+------------+----------------+-----+-----------+-----------+-----------------+-----------------+------+------+----------+--------+
|2015-09-05 13:30:00| AD|2015|            9|        248|           5|             Sat|   PM|         13|         13|                1|                1|    30|     0|         0|     UTC|
|2015-09-04 11:30:00| AD|2015|            9|        247|           4|             Fri|   AM|         11|         11|               11|               11|    30|     0|         0|     UTC|
|2018-09-01 00:01:00| AD|2018|            9|        244|         

In [49]:
rc.select(col('Date'),date_format(col('Date'),'d/MMM/yy hh:mm a')).show(truncate=False)

+-------------------+-----------------------------------+
|Date               |date_format(Date, d/MMM/yy hh:mm a)|
+-------------------+-----------------------------------+
|2015-09-05 13:30:00|5/Sep/15 01:30 PM                  |
|2015-09-04 11:30:00|4/Sep/15 11:30 AM                  |
|2018-09-01 00:01:00|1/Sep/18 12:01 AM                  |
|2015-09-05 12:45:00|5/Sep/15 12:45 PM                  |
|2015-09-05 13:00:00|5/Sep/15 01:00 PM                  |
|2015-09-05 10:55:00|5/Sep/15 10:55 AM                  |
|2015-09-04 18:00:00|4/Sep/15 06:00 PM                  |
|2015-09-05 13:00:00|5/Sep/15 01:00 PM                  |
|2015-09-05 11:30:00|5/Sep/15 11:30 AM                  |
|2016-05-01 00:25:00|1/May/16 12:25 AM                  |
|2015-09-05 14:00:00|5/Sep/15 02:00 PM                  |
|2015-09-05 11:00:00|5/Sep/15 11:00 AM                  |
|2015-09-05 03:00:00|5/Sep/15 03:00 AM                  |
|2015-09-05 12:50:00|5/Sep/15 12:50 PM                  |
|2015-09-03 13

# Joins

In [50]:
!wget https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2021-04-11 19:05:06--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                ]   5.57K  --.-KB/s    in 0s      

2021-04-11 19:05:06 (367 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [5699]



In [51]:
!mv /content/rows.csv?accessType=DOWNLOAD /content/police_stations.csv

In [52]:
ps = spark.read.csv('/content/police_stations.csv',header=True)

In [53]:
ps.show()

+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|            DISTRICT|    DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|        Headquarters|     Headquarters| 3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|                  18|       Near North|  1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 11

In [54]:
ps.cache()
ps.count()

24

In [55]:
ps.select(col('DISTRICT NAME'),col('DISTRICT')).distinct().show()

+-----------------+--------------------+
|    DISTRICT NAME|            DISTRICT|
+-----------------+--------------------+
|    South Chicago|                   4|
|        Near West|                  12|
| -87.6569725149)"|",Chicago,IL,6060...|
|         Harrison|                  11|
|      Shakespeare|                  14|
|          Gresham|                   6|
|    Grand Central|                  25|
|        Town Hall|                  19|
|     Headquarters|        Headquarters|
|      Rogers Park|                  24|
|        Wentworth|                   2|
|          Deering|                   9|
|          Calumet|                   5|
|       Near North|                  18|
|     Chicago Lawn|                   8|
|        Englewood|                   7|
|   Grand Crossing|                   3|
|      Morgan Park|                  22|
|   Jefferson Park|                  16|
|            Ogden|                  10|
+-----------------+--------------------+
only showing top

In [56]:
rc.cache()
rc.count()

7305272

In [57]:
rc.select(col('District')).distinct().show()

+--------+
|District|
+--------+
|     009|
|     012|
|     024|
|    null|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     005|
|     003|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
+--------+
only showing top 20 rows



In [58]:
from pyspark.sql.functions import lpad

In [59]:
rc = rc.withColumn('JOIN_DISTRICT',lpad(col('District'),3,'0'))

In [60]:
rc.show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|JOIN_DISTRICT|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOME

In [61]:
ps = ps.withColumn('JOIN_DISTRICT',lpad(col('DISTRICT'),3,'0'))
ps.show()

+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-------------+
|            DISTRICT|    DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|JOIN_DISTRICT|
+--------------------+-----------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-------------+
|        Headquarters|     Headquarters| 3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|          Hea|
|                  18|       Near North|  1160 N Larrabee St|Chicago|   IL|60610|http://

In [62]:
rc = rc.join(ps,rc.JOIN_DISTRICT==ps.JOIN_DISTRICT,'left')

In [63]:
rc.show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------+--------+--------------+--------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+-------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|JOIN_DISTRICT|DISTRICT| DISTRICT NAME|             ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|        

In [64]:
ps.columns

['DISTRICT',
 'DISTRICT NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION',
 'JOIN_DISTRICT']

In [65]:
rc = rc.drop('JOIN_DISTRICT','DISTRICT','ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'WEBSITE',
 'PHONE',
 'FAX',
 'TTY',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION')

In [66]:
rc.show()

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|Community Area|FBI Code|Year|          Updated On| DISTRICT NAME|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|  12|            61|     08B|2015|02/10/2018 03:50:...|       Deering|
|10224739|   HY411615|2015-09-04 11:30:00| 008XX N CENTRAL AVE|0870|             THEFT|      POCKET-PICKING|        

# Find the most frequently reported noncriminal activity

In [67]:
rc.select(col('Primary Type')).distinct().count()

36

In [68]:
rc.select(col('Primary Type')).distinct().orderBy(col('Primary Type')).show(rc.select(col('Primary Type')).distinct().count(),truncate=False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|ARSON                            |
|ASSAULT                          |
|BATTERY                          |
|BURGLARY                         |
|CONCEALED CARRY LICENSE VIOLATION|
|CRIM SEXUAL ASSAULT              |
|CRIMINAL DAMAGE                  |
|CRIMINAL SEXUAL ASSAULT          |
|CRIMINAL TRESPASS                |
|DECEPTIVE PRACTICE               |
|DOMESTIC VIOLENCE                |
|GAMBLING                         |
|HOMICIDE                         |
|HUMAN TRAFFICKING                |
|INTERFERENCE WITH PUBLIC OFFICER |
|INTIMIDATION                     |
|KIDNAPPING                       |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|NARCOTICS                        |
|NON - CRIMINAL                   |
|NON-CRIMINAL                     |
|NON-CRIMINAL (SUBJECT SPECIFIED) |
|OBSCENITY                        |
|OFFENSE INVOLVING CHILDREN 

In [69]:
rc.filter((col('Primary Type') == 'NON-CRIMINAL (SUBJECT SPECIFIED)') | (col('Primary Type') == 'NON-CRIMINAL') | (col('Primary Type') == 'NON - CRIMINAL')).show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|Community Area|FBI Code|Year|          Updated On| DISTRICT NAME|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+----+--------------+--------+----+--------------------+--------------+
|10233370|   HY421530|2015-09-13 10:45:00|  010XX S MENARD AVE|5114|      NON - CRIMINAL|   FOID - REVOCATION|           RESIDENCE|  true|   false|1513|  29|            25|      26|2015|02/10/2018 03:50:...|        Austin|
|10241515|   HY429346|2015-09-19 07:45:00|051XX N MILWAUKEE...|5114|      NON - CRIMINAL|   FOID - REVOCATIO

In [70]:
rc.filter((col('Primary Type') == 'NON-CRIMINAL (SUBJECT SPECIFIED)') | (col('Primary Type') == 'NON-CRIMINAL') | (col('Primary Type') == 'NON - CRIMINAL'))\
.groupby('Description')\
.count()\
.orderBy('count',ascending=False)\
.show(1)

+-------------+-----+
|  Description|count|
+-------------+-----+
|LOST PASSPORT|  111|
+-------------+-----+
only showing top 1 row



# Find the day of the week with most reported crime

In [71]:
rc.groupby(date_format(col('Date'),'E').alias('Day_week')).count().orderBy('count',ascending=False).show(1)

+--------+-------+
|Day_week|  count|
+--------+-------+
|     Fri|1099421|
+--------+-------+
only showing top 1 row



In [72]:
from pyspark.sql.functions import dayofweek

In [73]:
rc.groupby(dayofweek(col('Date')).alias('Day_week')).count().orderBy('count',ascending=False).show(1)

+--------+-------+
|Day_week|  count|
+--------+-------+
|       6|1099421|
+--------+-------+
only showing top 1 row



In [74]:
rc.groupby(date_format(col('Date'),'E').alias('Day_week'),dayofweek(col('Date')).alias('order_w')).count().orderBy('order_w',ascending=True).show()

+--------+-------+-------+
|Day_week|order_w|  count|
+--------+-------+-------+
|     Sun|      1| 990646|
|     Mon|      2|1031776|
|     Tue|      3|1044921|
|     Wed|      4|1051722|
|     Thu|      5|1041194|
|     Fri|      6|1099421|
|     Sat|      7|1045592|
+--------+-------+-------+



In [75]:
week = [x[0] for x in rc.groupby(date_format(col('Date'),'E').alias('Day_week'),dayofweek(col('Date')).alias('order_w')).count().orderBy('order_w',ascending=True).collect()]
week

['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

In [76]:
wcount = [x[2] for x in rc.groupby(date_format(col('Date'),'E').alias('Day_week'),dayofweek(col('Date')).alias('order_w')).count().orderBy('order_w',ascending=True).collect()]
wcount

[990646, 1031776, 1044921, 1051722, 1041194, 1099421, 1045592]

In [77]:
import plotly
import plotly.express as px
import pandas as pd

In [78]:
df = pd.DataFrame({'Day_of_week':week,'Count':wcount})
df

,Day_of_week,Count
0,Sun,990646
1,Mon,1031776
2,Tue,1044921
3,Wed,1051722
4,Thu,1041194
5,Fri,1099421
6,Sat,1045592


In [79]:
fig = px.line(
    data_frame=df,
    x="Day_of_week",
    y="Count",
    hover_name="Day_of_week"
)
fig.show()

In [80]:
df.sort_values('Count',inplace=True)

In [81]:
fig = px.line(
    data_frame=df,
    x="Day_of_week",
    y="Count",
    hover_name="Day_of_week",

)
fig.show()

# RDD

In [82]:
from pyspark import SparkContext

In [84]:
sparkc = SparkContext.getOrCreate()

In [89]:
psrdd = sparkc.textFile('/content/police_stations.csv')

In [88]:
psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [90]:
psheader = psrdd.first()

In [93]:
psdata = psrdd.filter(lambda line: line!=psheader)

In [94]:
psdata.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

In [95]:
psdata.count()

24

In [96]:
psdata.map(lambda line: line.split(',')).collect()

[['Headquarters',
  'Headquarters',
  '3510 S Michigan Ave',
  'Chicago',
  'IL',
  '60653',
  'http://home.chicagopolice.org',
  '',
  '',
  '',
  '1177731.401',
  '1881697.404',
  '41.83070169',
  '-87.62339535',
  '"(41.8307016873',
  ' -87.6233953459)"'],
 ['18',
  'Near North',
  '1160 N Larrabee St',
  'Chicago',
  'IL',
  '60610',
  'http://home.chicagopolice.org/community/districts/18th-district-near-north/',
  '312-742-5870',
  '312-742-5771',
  '312-742-5773',
  '1172080.029',
  '1908086.527',
  '41.90324165',
  '-87.64335214',
  '"(41.9032416531',
  ' -87.6433521393)"'],
 ['19',
  'Town Hall',
  '850 W Addison St',
  'Chicago',
  'IL',
  '60613',
  'http://home.chicagopolice.org/community/districts/19th-district-town-hall/',
  '312-744-8320',
  '312-744-4481',
  '312-744-8011',
  '1169730.744',
  '1924160.317',
  '41.94740046',
  '-87.65151202',
  '"(41.9474004564',
  ' -87.651512018)"'],
 ['20',
  'Lincoln',
  '5400 N Lincoln Ave',
  'Chicago',
  'IL',
  '60625',
  'http://

In [97]:
psdata.map(lambda line: line.split(',')).count()

24

# Select District, district name, address e Zip para a estacao policial 7

In [100]:
psdata.filter(lambda line: line.split(',')[0] == '7')\
.map(lambda line: (line.split(',')[0]\
                   ,line.split(',')[1]\
                   ,line.split(',')[2]\
                   ,line.split(',')[5]))\
                   .collect()

[('7', 'Englewood', '1438 W 63rd St', '60636')]

# Select District, district name, address e Zip para as estacoes policiais 10 e 11

In [101]:
psdata.filter(lambda line: line.split(',')[0] in ['10','11'])\
.map(lambda line: (line.split(',')[0]\
                   ,line.split(',')[1]\
                   ,line.split(',')[2]\
                   ,line.split(',')[5]))\
                   .collect()

[('10', 'Ogden', '3315 W Ogden Ave', '60623'),
 ('11', 'Harrison', '3151 W Harrison St', '60612')]